# Magister en Ciencia de Datos - UDD
## IAA (Inteligencia Artificial Aplicada)
### Desafio 1:  
Desarrollo de modelo de Recomendaciones para agente virtual <a href="https://cloud.ibm.com/docs/services/assistant?topic=assistant-getting-started#getting-started">Watson Assistan</a>. El modelo utilizará el paper: <a href="http://yifanhu.net/PUB/cf.pdf">Collaborative Filtering for Implicit Feedback Datasets</a>, de Yifan Hu, Yehuda Koren y Chris Volinsky. 
Para el desarrollo ocuparemos la API <a href="https://github.com/benfred/implicit">Implicit</a>, desarrollada por <a href="https://implicit.readthedocs.io/en/latest/">Ben Frederickson</a>(© Copyright 2017).

### Dataset:
Se utilizará un dataset de peliculas, series y programas en vivo para generar el modelos de recomendación. El alcance del modelo incluirá solamente un modelo para recomendar peliculas, pero se puede extender facilmente a Series y programas en vivo creando un identificador de contenido unico.

### Resultado:
Se crearan 3 bases de datos que contendran la siguiente informacion:
* Base con los codigos y los nombres de peliculas (db_item_lookup.csv)
* Base de recomendaciones de peliculas por usuario (db_user_score.csv)
* Base de recomendaciones de peliculas por item (db_item_score.csv)

By Hernan Rivera & Jose Pedro Berenguer

## Let's start :)

#### 1. Cargamos librerias para el desarrollo del modelo

In [1]:
import sys
import pandas as pd
import numpy as np
import scipy.sparse as sparse

from sklearn.preprocessing import MinMaxScaler

import implicit 

#### 2. Cargamos dataset "visual_meses" para el desafio.

In [2]:
raw_data = pd.read_csv('./data/visual_meses.csv', sep=";")

In [3]:
raw_data.head()

,ID_GRUPO,PAIS,ID_CLIENTE,TX_NOMBRE,TX_ESTUDIO,COD_FECHA_VISUALIZACION,FECHA_PAIS,TX_DISP_CATEGORIA_ORIGINAL,TX_DISP_MODELO_ORIGINAL,TX_DISP_TIPO_ORIGINAL,...,NUMERO_TEMPORADA,NUMERO_CAPITULO,NOMBRE_CAPITULO,NOMBRE_CAPITULO_ESP,DURACION_TOTAL,ESTUDIO,PROVEEDOR,TYPE,PROGRAM_ID,TX_NOMBRE_SERIE
0,526402,CHILE,85528236.0,National Treasure,Disney,2018-10-01 10:04:34,2018-10-01 10:04:34,tv,lg,TV,...,NaN,NaN,NaN,NaN,02:10:59,Disney,AMCO,Type 2,13594853.0,NaN
1,526402,CHILE,211677990.0,National Treasure,Disney,2019-01-04 05:55:14,2019-01-04 05:55:14,web,html5,html5,...,NaN,NaN,NaN,NaN,02:10:59,Disney,AMCO,Type 2,13594853.0,NaN
2,526402,CHILE,186668874.0,National Treasure,Disney,2018-12-03 19:08:05,2018-12-03 19:08:05,tablet,android,SM-P350,...,NaN,NaN,NaN,NaN,02:10:59,Disney,AMCO,Type 2,13594853.0,NaN
3,526402,CHILE,30095082.0,National Treasure,Disney,2019-02-06 16:42:47,2019-02-06 16:42:47,mobile,android,XT1097,...,NaN,NaN,NaN,NaN,02:10:59,Disney,AMCO,Type 2,13594853.0,NaN
4,526402,CHILE,204313488.0,National Treasure,Disney,2018-10-06 17:43:37,2018-10-06 17:43:37,mobile,android,moto g(6) play,...,NaN,NaN,NaN,NaN,02:10:59,Disney,AMCO,Type 2,13594853.0,NaN


In [4]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 187263 entries, 0 to 187262
Data columns (total 33 columns):
ID_GRUPO                       187263 non-null int64
PAIS                           187263 non-null object
ID_CLIENTE                     182247 non-null float64
TX_NOMBRE                      187263 non-null object
TX_ESTUDIO                     187263 non-null object
COD_FECHA_VISUALIZACION        187263 non-null object
FECHA_PAIS                     187263 non-null object
TX_DISP_CATEGORIA_ORIGINAL     179370 non-null object
TX_DISP_MODELO_ORIGINAL        179370 non-null object
TX_DISP_TIPO_ORIGINAL          179370 non-null object
TX_DISP_VERSION_ORIGINAL       31521 non-null object
COD_PARTNER_OPERACION          174492 non-null float64
TIPO_OPERACION                 174332 non-null object
PRECIO                         187263 non-null int64
MEDIO_PAGO_NOMBRE              174332 non-null object
FECHA_ULT_VIS                  131148 non-null object
ULT_VIS                   

* Las variables de interes indentificadas para el modelo son: ID_CLIENTE, TITULO, ID_GRUPO

In [5]:
raw_data = raw_data[["ID_CLIENTE", "ID_GRUPO", "TITULO", "CATEGORIA"]]

In [6]:
raw_data.CATEGORIA.value_counts()

Serie            123016
Pelicula          58845
Canal en Vivo      5402
Name: CATEGORIA, dtype: int64

#### 3. Analizamos el contendido de Peliculas y Series

In [7]:
raw_data[raw_data.CATEGORIA=="Pelicula"].groupby(["TITULO", "ID_GRUPO"]).size().head()

TITULO                          ID_GRUPO
10 años                         756331        4
10x10                           780952        1
12 Strong (aka Horse Soldiers)  771034       19
12 Years a Slave                596943       20
127 Hours                       575230      135
dtype: int64

In [8]:
raw_data[raw_data.CATEGORIA=="Serie"].groupby(["TITULO", "ID_GRUPO"]).size().head()

TITULO      ID_GRUPO
12 Monkeys  778016       2
            778017      30
            778035       4
            778036       2
            778049       4
dtype: int64

* Desde los datos se observa que solo se puede ocupar ID_GRUPO para Peliculas

#### 4. Generamos la lista de peliculas para ocuparlas en el sistema de recomendacion final

In [9]:
item_lookup = raw_data[raw_data.CATEGORIA=="Pelicula"][['ID_GRUPO', 'TITULO']].drop_duplicates() 
item_lookup['ID_GRUPO'] = item_lookup.ID_GRUPO.astype(str) 

In [10]:
item_lookup.head()

,ID_GRUPO,TITULO
0,526402,National Treasure
72,526403,Poohs Heffalump Movie
234,526406,Cars
406,526409,"Princess Diaries 2, The: Royal Engagement"
548,526422,Pirates of the Caribbean: The Curse of the Bla...


* Guardamos DF para base de datos y el sistema Watson Assistan

In [11]:
item_lookup.to_csv("./data/db_item_lookup.csv", index=False)

#### 5. Recuperamos el dataset que se utilizará para el modelo de peliculas

In [12]:
peliculas = raw_data[raw_data.CATEGORIA=="Pelicula"][["ID_CLIENTE", "ID_GRUPO"]]

#### 6. Eliminamos null y dejamos id_cliente como entero

In [13]:
peliculas = peliculas.dropna()

In [14]:
peliculas['ID_CLIENTE'] = peliculas.ID_CLIENTE.astype(int) 

##### 7. Contamos las veces que se han visto las peliculas

In [15]:
peliculas["CANTIDAD"] = 1

In [16]:
grouped_purchased = peliculas.groupby(['ID_CLIENTE', 'ID_GRUPO']).sum().reset_index() 

In [17]:
grouped_purchased.head()

,ID_CLIENTE,ID_GRUPO,CANTIDAD
0,16591302,528354,3
1,16591302,678597,1
2,16591302,775661,1
3,16599486,759487,1
4,16614228,776575,1


* El Dataset considera como valor minimo de agrupacion 1.

#### 8. Transformamos a dato categorico y reiniciamos secuencia 

In [18]:
grouped_purchased["ID_CLIENTE"] = grouped_purchased["ID_CLIENTE"].astype("category")
grouped_purchased["ID_GRUPO"] = grouped_purchased["ID_GRUPO"].astype("category")

In [19]:
grouped_purchased['ID_CLIENTE_ID'] = grouped_purchased['ID_CLIENTE'].cat.codes
grouped_purchased['ID_GRUPO_ID'] = grouped_purchased['ID_GRUPO'].cat.codes

In [20]:
grouped_purchased.head()

,ID_CLIENTE,ID_GRUPO,CANTIDAD,ID_CLIENTE_ID,ID_GRUPO_ID
0,16591302,528354,3,0,78
1,16591302,678597,1,0,1281
2,16591302,775661,1,0,2449
3,16599486,759487,1,1,2095
4,16614228,776575,1,2,2473


#### 9. Generamos lista de items y usuarios

In [21]:
item_idx = grouped_purchased[["ID_GRUPO", "ID_GRUPO_ID"]].drop_duplicates()

In [22]:
user_idx = grouped_purchased[["ID_CLIENTE", "ID_CLIENTE_ID"]].drop_duplicates()

#### 10.  La libreria implicit espera data configurada como matriz (item-user) y (user-item)

In [23]:
sparse_item_user = sparse.csr_matrix(
    (grouped_purchased['CANTIDAD'].astype(float), 
     (grouped_purchased['ID_GRUPO_ID'], 
      grouped_purchased['ID_CLIENTE_ID'])))

In [24]:
sparse_user_item = sparse.csr_matrix(
    (grouped_purchased['CANTIDAD'].astype(float), 
     (grouped_purchased['ID_CLIENTE_ID'], 
      grouped_purchased['ID_GRUPO_ID'])))

#### 11. Comenzamos el modelamiento

In [25]:
model = implicit.als.AlternatingLeastSquares(factors=20, regularization=0.1, iterations=20)

* Multiplicamos por un valor alpha con valor 15

In [26]:
alpha_val = 15
data_conf = (sparse_item_user * alpha_val).astype('double')

In [27]:
model.fit(data_conf)

100%|██████████| 20.0/20 [00:03<00:00,  6.00it/s]


#### 12. Buscamos recomendaciones sumilares por item (1 caso)

In [28]:
item_id = 1927 
n_similar = 3

In [29]:
user_vecs = model.user_factors
item_vecs = model.item_factors

In [30]:
def RecommendByItem(item_id, item_vecs, num_items=3):
    item_norms = np.sqrt((item_vecs * item_vecs).sum(axis=1))
    
    scores = item_vecs.dot(item_vecs[item_id]) / item_norms
    top_idx = np.argpartition(scores, -num_items)[-num_items:]
    similar = sorted(zip(top_idx, scores[top_idx] / item_norms[item_id]), key=lambda x: -x[1])
 
    stockcodes = []
    scores = []

    for item in similar:
        idx, score = item
        stockcodes.append(grouped_purchased.ID_GRUPO.loc[grouped_purchased.ID_GRUPO_ID == idx].iloc[0])
        scores.append(score)

    recommendations = pd.DataFrame({'ID_GRUPO_ID' : item_id, 'ID_GRUPO': stockcodes, 'Score': scores})
    
    return recommendations

In [31]:
RecommendByItem(1927, item_vecs, num_items=3)

,ID_GRUPO_ID,ID_GRUPO,Score
0,1927,746298,1.000000
1,1927,594323,0.727635
2,1927,771861,0.670516


#### 13. Buscamos recomendaciones sumilares por user (1 caso)

In [32]:
def RecommendByUser(user_id, sparse_user_item, user_vecs, item_vecs, num_items=3):
    """The same recommendation function we used before"""

    user_interactions = sparse_user_item[user_id,:].toarray()

    user_interactions = user_interactions.reshape(-1) + 1
    user_interactions[user_interactions > 1] = 0

    rec_vector = user_vecs[user_id,:].dot(item_vecs.T).toarray()

    min_max = MinMaxScaler()
    rec_vector_scaled = min_max.fit_transform(rec_vector.reshape(-1,1))[:,0]
    recommend_vector = user_interactions * rec_vector_scaled

    item_idx = np.argsort(recommend_vector)[::-1][:num_items]

    stockcodes = []
    scores = []

    for idx in item_idx:
        stockcodes.append(grouped_purchased.ID_GRUPO.loc[grouped_purchased.ID_GRUPO_ID == idx].iloc[0])
        scores.append(recommend_vector[idx])

    recommendations = pd.DataFrame({'ID_CLIENTE_ID' : user_id,'ID_GRUPO': stockcodes, 'Score': scores})

    return recommendations

In [33]:
user_vecs = sparse.csr_matrix(model.user_factors)
item_vecs = sparse.csr_matrix(model.item_factors)

user_id = 2025

recommendations = RecommendByUser(user_id, sparse_user_item, user_vecs, item_vecs)

print (recommendations)

   ID_CLIENTE_ID  ID_GRUPO     Score
0           2025    596060  1.000000
1           2025    532755  0.991613
2           2025    648707  0.984252


In [34]:
item_lookup[item_lookup.ID_GRUPO=="684678"]

,ID_GRUPO,TITULO
102924,684678,"Girl, Interrupted"


#### 14. Creamos base de recomendacion según Usuario.

In [35]:
def GetDB_Users(usuarios):
    dfs = []
    for user in usuarios.ID_CLIENTE_ID:
        dfs.append(RecommendByUser(user, sparse_user_item, user_vecs, item_vecs))
    return dfs   

In [36]:
df = GetDB_Users(user_idx)

In [37]:
pd.concat(df).to_csv("./data/db_tmp_user_score.csv", index=False)

In [38]:
df_tmp = pd.read_csv("./data/db_tmp_user_score.csv")

In [39]:
df_tmp.head()

,ID_CLIENTE_ID,ID_GRUPO,Score
0,0,540141,0.926875
1,0,779646,0.891710
2,0,532755,0.840386
3,1,591261,1.000000
4,1,528880,0.854312


* Creamos Dataset para cargar en la base

In [40]:
df_tb_user = pd.merge(df_tmp, user_idx, on='ID_CLIENTE_ID')[["ID_CLIENTE", "ID_GRUPO", "Score"]]

In [41]:
df_tb_user.head()

,ID_CLIENTE,ID_GRUPO,Score
0,16591302,540141,0.926875
1,16591302,779646,0.891710
2,16591302,532755,0.840386
3,16599486,591261,1.000000
4,16599486,528880,0.854312


In [42]:
df_tb_user.to_csv("./data/db_user_score.csv", index=False)

#### 15. Creamos base de Recomendacion para items

In [43]:
user_vecs = model.user_factors
item_vecs = model.item_factors

In [44]:
def GetDB_Item(items):
    dfs = []
    for item in items.ID_GRUPO_ID:
        dfs.append(RecommendByItem(item, item_vecs, 4))
    return dfs 

In [45]:
df = GetDB_Item(item_idx)

In [46]:
pd.concat(df).to_csv("./data/db_tmp_item_score.csv", index=False)

In [47]:
df_tmp = pd.read_csv("./data/db_tmp_item_score.csv")

In [48]:
df_tmp.head()

,ID_GRUPO_ID,ID_GRUPO,Score
0,78,528354,1.000000
1,78,578506,0.843519
2,78,714961,0.760762
3,78,777239,0.717573
4,1281,678597,1.000000


* Creamos dataset para cargar base de items

In [49]:
df_tb_item = pd.merge(df_tmp, item_idx, on='ID_GRUPO_ID')

In [50]:
del df_tb_item["ID_GRUPO_ID"]

In [51]:
df_tb_item.head()

,ID_GRUPO_x,Score,ID_GRUPO_y
0,528354,1.000000,528354
1,578506,0.843519,528354
2,714961,0.760762,528354
3,777239,0.717573,528354
4,678597,1.000000,678597


In [52]:
df_tb_item.rename(columns={'ID_GRUPO_x':'ID_GRUPO', 'ID_GRUPO_y':'ID_ITEM' }, inplace=True)

In [53]:
df_tb_item.head()

,ID_GRUPO,Score,ID_ITEM
0,528354,1.000000,528354
1,578506,0.843519,528354
2,714961,0.760762,528354
3,777239,0.717573,528354
4,678597,1.000000,678597


In [54]:
df_tb_item.to_csv("./data/db_item_score.csv", index=False)

#### Conclusiones:
Se puede obtener muy buenos resultados con este tipo de metodologia. Se puede mejorar utilizando en vez de las veces de aparacion, el porcentaje del tiempo que se vio la pelicula, con ello se agrega un elemento mas fino para efecto de recomendación.

Queda en liberar del lector expander el modelo para Series y Programas en vivo.